In [15]:
import requests

OSRM_URL = "http://localhost:5000"

import pandas as pd
df = pd.read_csv('train_cleaned.csv', index_col=0)

In [16]:
def get_osrm_route(start_lat, start_lng, end_lat, end_lng):
    """Fetch route details from OSRM based on start and end coordinates."""
    coordinates = f"{start_lng},{start_lat};{end_lng},{end_lat}"
    route_url = f"{OSRM_URL}/route/v1/driving/{coordinates}?overview=false"  # Change 'overview' if full route is needed

    response = requests.get(route_url)
    
    if response.status_code == 200:
        data = response.json()
        if data["routes"]:  # Ensure routes are returned
            return data["routes"][0]["distance"], data["routes"][0]["duration"]
    return None, None  # Return None if no valid response

# Filter the DataFrame to include only rows where start_city is 'NY'
ny_df = df[df['start_city'] == 'NY']
ny_df


,row_id,start_lng,start_lat,end_lng,end_lat,datetime,duration,missing_coords,start_city,end_city,...,bearing,hour,day_of_week,month,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
0,0,-73.783279,40.644718,-73.963799,40.681320,2015-01-09 00:51:48,1815.0,False,NY,NY,...,284.968645,0,4,1,0.000000e+00,1.000000,-0.433884,-0.900969,0.500000,8.660254e-01
2,2,-73.885292,40.773060,-74.000076,40.761360,2015-04-19 20:26:57,2620.0,False,NY,NY,...,262.401547,20,6,4,-8.660254e-01,0.500000,-0.781831,0.623490,0.866025,-5.000000e-01
4,4,-74.008728,40.714317,-74.006432,40.731720,2015-01-21 12:14:45,582.0,False,NY,NY,...,5.731066,12,2,1,1.224647e-16,-1.000000,0.974928,-0.222521,0.500000,8.660254e-01
5,5,-73.980431,40.738529,-73.988319,40.734810,2015-03-28 04:46:01,165.0,False,NY,NY,...,238.210547,4,5,3,8.660254e-01,0.500000,-0.974928,-0.222521,1.000000,6.123234e-17
7,7,-74.016220,40.710720,-74.002228,40.755859,2015-04-27 11:36:19,726.0,False,NY,NY,...,13.263135,11,0,4,2.588190e-01,-0.965926,0.000000,1.000000,0.866025,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145996,145996,-74.017975,40.706478,-74.002213,40.686069,2015-05-27 23:38:58,713.0,False,NY,NY,...,149.547289,23,2,5,-2.588190e-01,0.965926,0.974928,-0.222521,0.500000,-8.660254e-01
145997,145997,-73.968781,40.757835,-73.951843,40.769596,2015-01-07 02:58:34,308.0,False,NY,NY,...,47.591480,2,2,1,5.000000e-01,0.866025,0.974928,-0.222521,0.500000,8.660254e-01
145998,145998,-74.005699,40.740265,-73.982620,40.777214,2015-09-07 00:17:32,677.0,False,NY,NY,...,25.398119,0,0,9,0.000000e+00,1.000000,0.000000,1.000000,-1.000000,-1.836970e-16
145999,145999,-73.972466,40.754219,-73.974335,40.759819,2015-11-09 15:13:00,435.0,False,NY,NY,...,345.760267,15,0,11,-7.071068e-01,-0.707107,0.000000,1.000000,-0.500000,8.660254e-01


In [17]:
ny_df["osrm_distance"], ny_df["osrm_duration"] = zip(*ny_df.apply(lambda row: get_osrm_route(
    row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]), axis=1))

/tmp/ipykernel_28664/1992662405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_df["osrm_distance"], ny_df["osrm_duration"] = zip(*ny_df.apply(lambda row: get_osrm_route(
/tmp/ipykernel_28664/1992662405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_df["osrm_distance"], ny_df["osrm_duration"] = zip(*ny_df.apply(lambda row: get_osrm_route(


In [18]:
ny_df[['osrm_duration', 'osrm_distance']]

,osrm_duration,osrm_distance
0,1626.5,19781.3
2,1162.9,15257.3
4,347.9,2238.5
5,139.1,1083.7
7,646.2,5724.5
...,...,...
145996,510.2,5605.7
145997,252.0,2574.5
145998,642.5,5089.3
145999,115.9,915.6


In [19]:
ny_df[['osrm_duration', 'osrm_distance']].isnull().sum()

osrm_duration    1
osrm_distance    1
dtype: int64

In [20]:
ny_df[['osrm_duration', 'osrm_distance']].describe()

,osrm_duration,osrm_distance
count,101999.000000,101999.000000
mean,460.233656,4640.655838
std,373.729009,5375.094113
min,0.000000,0.000000
25%,214.100000,1696.700000
50%,344.000000,2815.300000
75%,577.900000,5144.000000
max,6175.500000,106959.600000


In [21]:
ny_df = ny_df[(ny_df['osrm_distance'] > 0) & (ny_df['osrm_duration'] > 0)]
ny_df[['osrm_duration', 'osrm_distance']]

,osrm_duration,osrm_distance
0,1626.5,19781.3
2,1162.9,15257.3
4,347.9,2238.5
5,139.1,1083.7
7,646.2,5724.5
...,...,...
145996,510.2,5605.7
145997,252.0,2574.5
145998,642.5,5089.3
145999,115.9,915.6


In [23]:
ny_df[['osrm_duration', 'osrm_distance']].describe()

,osrm_duration,osrm_distance
count,101423.000000,101423.000000
mean,462.847408,4667.010867
std,373.171315,5378.914333
min,0.100000,0.100000
25%,215.900000,1713.350000
50%,345.900000,2832.500000
75%,579.800000,5167.250000
max,6175.500000,106959.600000


In [22]:
ny_df.to_csv('ny_df_osmr.csv', index=False)

test


In [8]:
df = pd.read_csv('test_cleaned.csv', index_col=0)
ny_df = df[df['start_city'] == 'NY']
ny_df["osrm_distance"], ny_df["osrm_duration"] = zip(*ny_df.apply(lambda row: get_osrm_route(
    row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]), axis=1))
ny_df[['osrm_duration', 'osrm_distance']]

/tmp/ipykernel_28664/1191485327.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_df["osrm_distance"], ny_df["osrm_duration"] = zip(*ny_df.apply(lambda row: get_osrm_route(
/tmp/ipykernel_28664/1191485327.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_df["osrm_distance"], ny_df["osrm_duration"] = zip(*ny_df.apply(lambda row: get_osrm_route(


,osrm_duration,osrm_distance
0,789.6,6651.1
2,1568.2,25540.7
4,1081.7,14268.8
6,501.1,3722.8
8,753.8,7934.6
...,...,...
29995,328.6,3047.3
29996,1696.4,26935.1
29997,131.9,965.1
29998,647.6,5046.3


In [9]:
ny_df[['osrm_duration', 'osrm_distance']].isnull().sum()

osrm_duration    0
osrm_distance    0
dtype: int64

In [10]:
ny_df[['osrm_duration', 'osrm_distance']].describe()

,osrm_duration,osrm_distance
count,20000.000000,20000.000000
mean,460.334565,4646.332140
std,376.350542,5423.820289
min,0.000000,0.000000
25%,213.500000,1696.700000
50%,343.050000,2817.900000
75%,576.125000,5136.350000
max,3468.900000,59048.700000


In [12]:
ny_df = ny_df[(ny_df['osrm_distance'] > 0) & (ny_df['osrm_duration'] > 0)]

In [13]:
ny_df[['osrm_duration', 'osrm_distance']]

,osrm_duration,osrm_distance
0,789.6,6651.1
2,1568.2,25540.7
4,1081.7,14268.8
6,501.1,3722.8
8,753.8,7934.6
...,...,...
29995,328.6,3047.3
29996,1696.4,26935.1
29997,131.9,965.1
29998,647.6,5046.3


In [14]:
ny_df.to_csv('ny_df_osmr_test.csv', index=False)